In [6]:
import os
import numpy as np
from tensorflow.keras.preprocessing import image

def load_test_images(test_dir, target_size=(224, 224)):
    test_images = []
    filenames = []

    # Allowed formats
    valid_exts = ('.jpg', '.jpeg', '.png', '.bmp', '.tif', '.tiff')

    for fname in os.listdir(test_dir):
        if fname.lower().endswith(valid_exts):
            try:
                img_path = os.path.join(test_dir, fname)
                img = image.load_img(img_path, target_size=target_size)
                img_array = image.img_to_array(img) / 255.0
                test_images.append(img_array)
                filenames.append(fname)
            except Exception as e:
                print(f"⚠️ Skipping {fname} due to error: {e}")

    test_images = np.array(test_images)
    print(f"✅ Loaded {len(test_images)} images from {test_dir}")
    return test_images, filenames


# ---- USAGE ----
test_dir = r"C:\Users\Akhila\digita_pathology\data\Test cases"
test_images, filenames = load_test_images(test_dir)

print("Shape of test_images:", test_images.shape)
print("First 5 files:", filenames[:5])


✅ Loaded 197 images from C:\Users\Akhila\digita_pathology\data\Test cases
Shape of test_images: (197, 224, 224, 3)
First 5 files: ['000001_03_01_088.png', '000017_02_01_016.png', '000019_01_01_021.png', '000019_02_01_025.png', '000019_03_01_025.png']


In [7]:
# Assuming you already trained your model
# and it was named "model"

predictions = model.predict(test_images)

# Convert predictions (softmax) to class indices
predicted_indices = np.argmax(predictions, axis=1)

# Get class names from training dataset
class_names = train_ds.class_names  # ['Benign cases', 'Malignant cases', 'Normal cases']

# Map indices to class labels
predicted_labels = [class_names[i] for i in predicted_indices]

# Show first 10 predictions
for fname, label in zip(filenames[:10], predicted_labels[:10]):
    print(f"{fname} → {label}")


NameError: name 'model' is not defined

In [5]:
import os
import numpy as np
from tensorflow.keras.preprocessing import image

test_dir = r"C:\Users\Akhila\digita_pathology\data\Test cases"

test_images = []
filenames = []

for fname in os.listdir(test_dir):
    if fname.lower().endswith(('.jpg', '.jpeg', '.png')):
        img_path = os.path.join(test_dir, fname)
        img = image.load_img(img_path, target_size=(224, 224))
        img_array = image.img_to_array(img) / 255.0
        test_images.append(img_array)
        filenames.append(fname)

test_images = np.array(test_images)
print("Loaded test images:", test_images.shape)


Loaded test images: (197, 224, 224, 3)


In [11]:
from tensorflow.keras.models import load_model

# Load model file (replace with your actual saved filename)
model = load_model("lung_cancer_model.h5")

# Now you can predict
predictions = model.predict(test_images)


7/7 ━━━━━━━━━━━━━━━━━━━━ 18s 2s/step


In [10]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras import layers, models

num_classes = 3  # Benign, Malignant, Normal

base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224,224,3))
base_model.trainable = False  # freeze all convolutional layers

# Add a simple classifier on top
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(num_classes, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Train only the top classifier (should be fast)
history = model.fit(train_ds, epochs=3)  # 3–5 epochs enough for a demo

# Save it
model.save("lung_cancer_model.h5")


Epoch 1/3
35/35 ━━━━━━━━━━━━━━━━━━━━ 95s 2s/step - accuracy: 0.8876 - loss: 0.2919
Epoch 2/3
35/35 ━━━━━━━━━━━━━━━━━━━━ 89s 3s/step - accuracy: 1.0000 - loss: 0.0015
Epoch 3/3
35/35 ━━━━━━━━━━━━━━━━━━━━ 64s 2s/step - accuracy: 1.0000 - loss: 0.0011
